# Pixel Baseline

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader
import time
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from datasets import *

b16 = [32, 41, 51, 66, 76, 84, 89, 94, 105, 119, 136, 140, 159, 170, 187, 192]
optimb11 = []

In [ ]:
inp = []
out = []
for file in os.listdir(in_file):
    base = file[7:-4]
    try:
        clss = np.array(Image.open(out_file + base + "_annotate.png"))
        inp_im = np.load(in_file + file)
        print(base, end="\r")
        rows, cols = np.nonzero(clss)
        for r, c in zip(rows, cols):
            inp.append(inp_im[r, c])
            out.append(clss[r, c])
    except:
        continue

vinf = "../valid/full_inp/"
voutf = "../vlaid/gt/"
valinp = []
valout = []
for file in os.listdir(vinf):
    base = file[7:-4]
    inp_im = np.load(vinf + file)
    try:
        clss = np.array(Image.open(voutf + base + "_annotate.png"))
        print(base, end="\r")
        rows, cols = np.nonzero(clss)
        for r, c in zip(rows, cols):
            valinp.append(inp_im[r, c])
            valout.append(clss[r, c])
    except:
        continue

inp = np.array(inp)[:, b16]
valinp = np.array(valinp)[:, b16]
out = np.array(out, dtype="uint8") - 1
valout = np.array(valout, dtype="uint8") - 1

cls = MLPClassifier(hidden_layer_sizes=(16), max_iter=5, activation='tanh', learning_rate_init=0.001)
cls.fit(inp, out)

m = torch.load("../models/pixel_baseline.pth")
cls.coefs_[0] = m["0.weight"].cpu().numpy().T
cls.coefs_[1] = m["2.weight"].cpu().numpy().T
cls.intercepts_[0] = m["0.bias"].cpu().numpy().T
cls.intercepts_[1] = m["2.bias"].cpu().numpy().T

labels = ["Stroma", "Benign", "Other", "Malignant"]
colors = ["purple", "g", "b", "r"]

def roc_VIT(yp, yt):
    total = []
    for cl in range(len(labels)): 
        testy = yt == cl
        lr_prob = yp[:, cl]
        
        lr_auc = roc_auc_score(testy, lr_prob)
        total.append(lr_auc)
        lr_fpr, lr_tpr, prob_th = roc_curve(testy, lr_prob)
        
        optim = 0.5
        mn = 1
        plt.plot(lr_fpr, lr_tpr, color=colors[cl], label=labels[cl] + ", AUC: " + str(round(lr_auc, 4)))
    plt.title("ROC curve 4 class")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.savefig("plot.png")
    plt.show()
    
    yp = np.argmax(yp, axis=1)
    ConfusionMatrixDisplay.from_predictions(yt, yp, display_labels=labels)
    plt.title("Confusion Matrix")
    plt.savefig("cf.png")
    plt.show()

roc_VIT(cls.predict_proba(valinp), valout)

# 16 Band Models

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
from spectral import open_image

from arch import *

import torchvision.transforms as transforms
import torch.nn as nn
import torch

In [ ]:
# 3 EPI STROMA
colors = ["purple", "g", "b", "r"]
labels = np.array([[0, 0, 0], [124,55,144], [0, 255, 0], [0, 0, 255], [255, 0, 0]], dtype="uint8")

def forward_all(model, im, amide_ind, bkgrd, ps=25, bs=40):
    out = np.zeros((im.shape[-2], im.shape[-1], 3), dtype="uint8")
    annotate =  np.zeros((im.shape[-2], im.shape[-1]), dtype="uint8")
    pad = (ps//2, ps//2, ps//2, ps//2)
    im = F.pad(im, pad, "constant").float()
    input_num_band = 11
    generator = VisionTransformer(ps, input_num_band, n_class=3, flatten_dim=64)
    generator.add_lin_proj("saved_models_drop/cascade/optim_preembed.pth", input_num_band)
#     generator = ACD_VGG_BN_256(16, 4)
#     generator = models.resnet18(pretrained=False)
#     generator.conv1 = nn.Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#     generator.fc = nn.Linear(in_features=512, out_features=4, bias=True)
    generator.load_state_dict(torch.load(model))
    generator.to("cuda")
    generator.eval()
    

    curr_bs = 0
    index_dict = {}
    inp = None
    for x in range(out.shape[0]):
        print(x, end="\r")
        inpx = x + ps//2
        for y in range(out.shape[1]):
            inpy = y + ps//2
            if im[0, amide_ind, inpx, inpy] < bkgrd:
                annotate[x, y] = 0
                out[x, y] = labels[0]
            else:
                if curr_bs == 0:
                    inp = im[:, :, inpx - ps//2:inpx + ps//2 + 1, inpy - ps//2:inpy + ps//2 + 1]
                else:
                    inp = torch.cat([
                        inp, 
                        im[:, :, inpx - ps//2:inpx + ps//2 + 1, inpy - ps//2:inpy + ps//2 + 1]
                    ], axis=0)
                index_dict[curr_bs] = (x, y)
                curr_bs += 1
            
            if curr_bs == bs or (y+1 == out.shape[1] and curr_bs != 0):
                t1 = np.array(F.softmax(generator(inp.cuda()), dim=-1).cpu().detach())
                pred = np.argmax(t1, axis=-1) + 1
                for i in range(len(pred)):
                    r, c = index_dict[i]
                    annotate[r, c] = pred[i]
                    out[r, c] = labels[pred[i]]
                curr_bs = 0
                index_dict = {}
                inp = None
    return np.array(out, dtype="uint8"), np.array(annotate, dtype="uint8")
    
def forward_patch_VIT(model, im, out, ps=25):
    predicted = []
    actual = []
    batch = 32

    pad = (ps//2, ps//2, ps//2, ps//2)
    im = F.pad(im, pad, "constant").float()
    input_num_band = im.shape[1]
    generator = VisionTransformer(ps, input_num_band, n_class=3, flatten_dim=64)
#     generator.add_lin_proj("saved_models_drop/cascade/optim_preembed.pth", input_num_band)
#     generator = ACD_VGG_BN_256(16, 4)
#     generator = models.resnet18(pretrained=False)
#     generator.conv1 = nn.Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#     generator.fc = nn.Linear(in_features=512, out_features=4, bias=True)
    generator.load_state_dict(torch.load(model))
    generator.to("cuda")
    generator.eval()
    r, c = np.nonzero(out)
    inp = None
    count = 0
    for x, y in zip(r, c):
        count += 1
        if inp is None:
            inp = im[:, :, x - ps//2:x + ps//2 + 1, y - ps//2:y + ps//2 + 1]
        else:
            inp = torch.cat([inp, im[:, :, x - ps//2:x + ps//2 + 1, y - ps//2:y + ps//2 + 1]], axis=0)
        actual.append(out[x, y] - 1)
        
        if count % batch == 0:
            predicted += list(np.array(F.softmax(generator(inp.cuda())).cpu().detach()))
            del inp
            inp = None
    predicted += list(np.array(F.softmax(generator(inp.cuda())).cpu().detach()))
    del im
    return predicted, actual

def get_tensor_VIT(I_name):
    img = np.load(I_name)
    trans2 = transforms.ToTensor()
    img = torch.unsqueeze(trans2(img), dim=0)
    
    return img

def run_projection(model_f, inp, outf, amide_ind, bkgrd):
    for file in os.listdir(inp):
        print(file)
        im = get_tensor_VIT(inp + file)
        print(outf + file[:-3] + "png")
        out, annotate = forward_all(model_f, im, amide_ind, bkgrd)
        Image.fromarray(annotate).save(outf + file[7:-4] + "_annotate.png")
        Image.fromarray(out).save(outf + file[:-3] + "png")
        
    return
        

def roc_VIT(model_f, inp_f, gt_f, pixel=False):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve
    from sklearn.metrics import roc_auc_score
    from matplotlib import pyplot
    
    yt = []
    yp = []
    labels = ["Stroma", "Benign", "Other", "Malignant"]
    
    for file in os.listdir(inp_f):
        gtim = None
        try:
            print(file)
            gtim = np.array(Image.open(gt_f + file[7:-4] + "_annotate.png"))
        except:
            continue
        im = get_tensor_VIT(inp_f + file)
        if pixel:
            pred, true = forward_pixel(model_f, im, gtim)
        else:
            pred, true = forward_patch_VIT(model_f, im, gtim)
        yp += pred
        yt += true
    
    yp = np.array(yp)
    yt = np.array(yt)
    yt[yt == 3] = 1 # remove for 4 class
    labels = ["stroma", "epithelium", "other"]
    
    total = []
    for cl in range(len(labels)): 
        testy = yt == cl
        lr_prob = yp[:, cl]
        
        lr_auc = roc_auc_score(testy, lr_prob)
        total.append(lr_auc)
        lr_fpr, lr_tpr, _ = roc_curve(testy, lr_prob)
        pyplot.plot(lr_fpr, lr_tpr, color=colors[cl], label=labels[cl] + ", AUC: " + str(round(lr_auc, 4)))
    plt.title("ROC curve 3 class")
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    pyplot.legend()
    pyplot.savefig("plot3.png")
    pyplot.show()
    
    yp = np.argmax(yp, axis=1)
    from sklearn.metrics import ConfusionMatrixDisplay
    ConfusionMatrixDisplay.from_predictions(yt, yp, display_labels=labels)
    plt.title("Confusion Matrix")
    pyplot.savefig("cm.png")
    pyplot.show()
    
    print(model_f, sum(total)/len(total), min(total))

    
proj_model = "../models/s1_optim.pth"
inf = "../ir_im/optim_bands/"
outf = "../proj/epistr/"
amide_ind = [-2, 187, 3]
bkgrd = [0.1, 0.1, 0.03]
j = 1
run_projection(proj_model, inf, outf, amide_ind[j], bkgrd[j])

inf = "../valid/optim_in/"
gtf = "../valid/gt/"
roc_VIT(proj_model, inf, gtf)

In [ ]:
# BENIGN MALIGNANT
colors = ["g", "r"]
labels = np.array([[0, 0, 0], [0, 255, 0], [255, 0, 0]], dtype="uint8")

def forward_all(model, im, nucf, ps=25, bs=40):
    out = np.zeros((im.shape[-2], im.shape[-1], 3), dtype="uint8")
    annotate =  np.zeros((im.shape[-2], im.shape[-1]), dtype="uint8")
    pad = (ps//2, ps//2, ps//2, ps//2)
    im = F.pad(im, pad, "constant").float()
    generator = VisionTransformer(ps, 11, n_class=2, flatten_dim=64)
    generator.add_lin_proj("saved_models_drop/cascade/optim_preembed.pth")
#     generator = ACD_VGG_BN_256(16, 4)
#     generator = models.resnet18(pretrained=False)
#     generator.conv1 = nn.Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#     generator.fc = nn.Linear(in_features=512, out_features=4, bias=True)
    generator.load_state_dict(torch.load(model))
    generator.to("cuda")
    generator.eval()
    

    curr_bs = 0
    index_dict = {}
    inp = None
    for x in range(out.shape[0]):
        print(x, end="\r")
        inpx = x + ps//2
        for y in range(out.shape[1]):
            inpy = y + ps//2
            if nucf[x, y] != 2:
                annotate[x, y] = 0
                out[x, y] = labels[0]
            else:
                if curr_bs == 0:
                    inp = im[:, :, inpx - ps//2:inpx + ps//2 + 1, inpy - ps//2:inpy + ps//2 + 1]
                else:
                    inp = torch.cat([
                        inp, 
                        im[:, :, inpx - ps//2:inpx + ps//2 + 1, inpy - ps//2:inpy + ps//2 + 1]
                    ], axis=0)
                index_dict[curr_bs] = (x, y)
                curr_bs += 1
            
            if curr_bs == bs or (y+1 == out.shape[1] and curr_bs != 0):
                t1 = np.array(F.softmax(generator(inp.cuda()), dim=-1).cpu().detach())
                pred = (t1[:, -1] > 0.9999).astype("int") + 1
                for i in range(len(pred)):
                    r, c = index_dict[i]
                    annotate[r, c] = pred[i]
                    out[r, c] = labels[pred[i]]
                curr_bs = 0
                index_dict = {}
                inp = None
    return np.array(out, dtype="uint8"), np.array(annotate, dtype="uint8")

def forward_patch_VIT(model, im, out, ps=25):
    predicted = []
    actual = []
    batch = 32

    pad = (ps//2, ps//2, ps//2, ps//2)
    im = F.pad(im, pad, "constant").float()
    
    generator = VisionTransformer(ps, im.shape[1], n_class=2, flatten_dim=64)
#     generator.add_lin_proj("saved_models_drop/cascade/optim_preembed.pth")
#     generator = VisionTransformer(ps, 226, n_class=2, flatten_dim=64)
#     generator = ACD_VGG_BN_256(16, 4)
#     generator = models.resnet18(pretrained=False)
#     generator.conv1 = nn.Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#     generator.fc = nn.Linear(in_features=512, out_features=4, bias=True)
    
    generator.load_state_dict(torch.load(model))
    generator.to("cuda")
    generator.eval()
    r, c = np.nonzero(out)
    inp = None
    count = 0
    for x, y in zip(r, c):
        if out[x,y] not in [2, 4]:
            continue
        count += 1
        if inp is None:
            inp = im[:, :, x - ps//2:x + ps//2 + 1, y - ps//2:y + ps//2 + 1]
        else:
            inp = torch.cat([inp, im[:, :, x - ps//2:x + ps//2 + 1, y - ps//2:y + ps//2 + 1]], axis=0)
        actual.append(int(out[x, y] == 4))
        
        if count % batch == 0:
            result = F.softmax(generator(inp.cuda())).cpu().detach()
            predicted += list(np.array(result))
            del inp
            inp = None
    predicted += list(np.array(F.softmax(generator(inp.cuda())).cpu().detach()))
    del im
    return predicted, actual

def get_tensor_VIT(I_name):
    img = np.load(I_name)
    trans2 = transforms.ToTensor()
    img = torch.unsqueeze(trans2(img), dim=0)
    
    return img

def run_projection(model_f, inp, outf, nucf):
    bands = [32, 41, 51, 66, 76, 84, 89, 94, 105, 119, 136, 140, 159, 170, 187, 192]
    for file in os.listdir(inp)[:]: #################################
        base = file[7:-4]
        if base != "L11":
            continue
        ann_end = "_annotate.png"
        print(base)
        im = get_tensor_VIT(inp + file)[:, :]
        n = np.array(Image.open("%s%s%s" %(nucf, base, ann_end)))
        print(outf + file[:-3] + "png")
        out, annotate = forward_all(model_f, im, n)
        Image.fromarray(annotate).save(outf + base + ann_end)
        Image.fromarray(out).save(outf + file[:-3] + "png")
        
    return
        

def roc_VIT(model_f, inp_f, gt_f, pixel=False):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve
    from sklearn.metrics import roc_auc_score
    from matplotlib import pyplot
    
    yt = []
    yp = []
    labels = ["Benign", "Malignant"]
    
    for file in os.listdir(inp_f):
        gtim = None
        try:
            print(file)
            gtim = np.array(Image.open(gt_f + file[7:-4] + "_annotate.png"))
        except:
            continue
        im = get_tensor_VIT(inp_f + file)
        if pixel:
            pred, true = forward_pixel(model_f, im, gtim)
        else:
            pred, true = forward_patch_VIT(model_f, im, gtim)
        yp += pred
        yt += true
    
    yp = np.array(yp)[:,-1]
    yt = np.array(yt)
    
    total = []
    cl=1
    testy = yt == cl
    lr_prob = yp

    lr_auc = roc_auc_score(testy, lr_prob)
    total.append(lr_auc)
    lr_fpr, lr_tpr, prob_th = roc_curve(testy, lr_prob)

    
    pyplot.plot(lr_fpr, lr_tpr, color=colors[cl], label=labels[cl] + ", AUC: " + str(round(lr_auc, 4)))
    plt.title("ROC curve 1 class")
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    pyplot.legend()
    pyplot.savefig("plot.png")
    pyplot.show()
    
    
    fp = []
    max_acc = 0
    prob_f = 0
    for prob in prob_th:
        acc = sum((yp > prob) == testy) / len(yp)
        if acc > max_acc:
            fp = yp > prob
            max_acc = acc
            prob_f = prob
    print(prob_f)
    
    from sklearn.metrics import ConfusionMatrixDisplay
    ConfusionMatrixDisplay.from_predictions(yt, fp.astype("int"), display_labels=labels)
    plt.title("Confusion Matrix")
    pyplot.savefig("cm.png")
    pyplot.show()
    
    print(model_f, sum(total)/len(total), min(total))

    
proj_model = ",,.models/s2_optim.pth"
nucf = "../proj/epistr/"
inf = "../ir_im/optim_bands/"
outf = "../proj/malign/"
run_projection(proj_model, inf, outf)

inf = "../valid/optim_in/"
gtf = "../valid/gt/"
roc_VIT(proj_model, inf, gtf)

In [ ]:
# SMOOTH MUSCLE & pros_hyper
colors = ["g", "r"]
labels = np.array([[0, 0, 0], [0, 255, 0], [255, 255, 0]], dtype="uint8")

def forward_all(model, im, nucf, ps=25, bs=40):
    print(im.shape)
    bkgrd = 0.2
    out = np.zeros((im.shape[-2], im.shape[-1], 3), dtype="uint8")
    annotate =  np.zeros((im.shape[-2], im.shape[-1]), dtype="uint8")
    pad = (ps//2, ps//2, ps//2, ps//2)
    im = F.pad(im, pad, "constant").float()
    generator = VisionTransformer(ps, 11, n_class=2, flatten_dim=64)
    generator.add_lin_proj("saved_models_drop/cascade/optim_preembed.pth")
#     generator = ACD_VGG_BN_256(16, 4)
#     generator = models.resnet18(pretrained=False)
#     generator.conv1 = nn.Conv2d(16, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
#     generator.fc = nn.Linear(in_features=512, out_features=4, bias=True)
    generator.load_state_dict(torch.load(model))
    generator.to("cuda")
    generator.eval()
    

    curr_bs = 0
    index_dict = {}
    inp = None
    for x in range(out.shape[0]):
        print(x, end="\r")
        inpx = x + ps//2
        for y in range(out.shape[1]):
            inpy = y + ps//2
            if nucf[x, y] != 1: # 
                annotate[x, y] = 0
                out[x, y] = labels[0]
            else:
                if curr_bs == 0:
                    inp = im[:, :, inpx - ps//2:inpx + ps//2 + 1, inpy - ps//2:inpy + ps//2 + 1]
                else:
                    inp = torch.cat([
                        inp, 
                        im[:, :, inpx - ps//2:inpx + ps//2 + 1, inpy - ps//2:inpy + ps//2 + 1]
                    ], axis=0)
                index_dict[curr_bs] = (x, y)
                curr_bs += 1
            
            if curr_bs == bs or (y+1 == out.shape[1] and curr_bs != 0):
                t1 = np.array(F.softmax(generator(inp.cuda()), dim=-1).cpu().detach())
                pred = np.argmax(t1, axis=-1) + 1
                for i in range(len(pred)):
                    r, c = index_dict[i]
                    annotate[r, c] = pred[i]
                    out[r, c] = labels[pred[i]]
                curr_bs = 0
                index_dict = {}
                inp = None
    return np.array(out, dtype="uint8"), np.array(annotate, dtype="uint8")


def get_tensor_VIT(I_name):
    img = np.load(I_name)
    trans2 = transforms.ToTensor()
    img = torch.unsqueeze(trans2(img), dim=0)
    
    return img

def run_projection(model_f, inp, outf, nucf):
    flag = False
    for file in os.listdir(inp):
        base = file[7:-4]
        if not flag and base != "H6":
            continue
        flag = True
        ann_end = "_annotate.png"
        im = get_tensor_VIT(inp + file)
        n = np.array(Image.open("%s%s%s" %(nucf, base, ann_end)))
        print(outf + file[:-3] + "png")
        out, annotate = forward_all(model_f, im, n)
        Image.fromarray(annotate).save(outf + base + ann_end)
        Image.fromarray(out).save(outf + file[:-3] + "png")
        
    return

inf = "../ir_im/full_bands/"
outf = "../proj/smooth_muscle/"
nucf = "../proj/epistr/" # "proj/malign"
proj_model = "models/s3_full_spectra.pth" # "models/s4_full_spectra.pth"
run_projection(proj_model, inf, outf, nucf)